Jasne! W **Spark MLlib** (API DataFrame: `pyspark.ml`) masz komplet klocków do budowy całego pipeline’u: od przygotowania danych, przez modele, aż po strojenie i ewaluację. Poniżej „mapa terenu” pod Spark 3.x (to, czego realnie użyjesz na co dzień).

---

## 1) Przygotowanie danych (Transformers)

* **Wejście/cechy:**

  * `VectorAssembler` – sklejanie wielu kolumn num. do jednej `features`.
  * `StringIndexer`, `OneHotEncoder` – kategoryczne → indeksy → wektory.
  * `Imputer` – uzupełnianie braków (mean/median).
* **Skalowanie / normalizacja:**

  * `StandardScaler`, `MinMaxScaler`, `MaxAbsScaler`, `Normalizer`.
* **Inżynieria cech:**

  * `Bucketizer`, `QuantileDiscretizer`, `FeatureHasher`, `PolynomialExpansion`.
  * Tekst: `Tokenizer`, `RegexTokenizer`, `StopWordsRemover`,
    `CountVectorizer`, `HashingTF`, `IDF`, `NGram`, `Word2Vec`.
* **Selekcja/Redukcja:**

  * `ChiSqSelector`, `PCA`.

---

## 2) Modele (Estimators)

### Klasyfikacja

* `LogisticRegression` (bin./wieloklasowa przez `OneVsRest`)
* `LinearSVC` (SVM liniowy, binary)
* `DecisionTreeClassifier`, `RandomForestClassifier`, `GBTClassifier`
* `NaiveBayes`
* `MultilayerPerceptronClassifier` (prosta sieć)
* `FMClassifier` (Factorization Machines)

### Regresja

* `LinearRegression`, `GeneralizedLinearRegression`
* `DecisionTreeRegressor`, `RandomForestRegressor`, `GBTRegressor`
* `AFTSurvivalRegression`
* `IsotonicRegression`
* `FMRegressor`

### Klasteryzacja / tematy

* `KMeans`, `BisectingKMeans`, `GaussianMixture`
* `LDA` (topic modeling)

### Rekomendacje

* `ALS` (collaborative filtering)

---

## 3) Ewaluacja i metryki

* `BinaryClassificationEvaluator`, `MulticlassClassificationEvaluator`
* `RegressionEvaluator`, `ClusteringEvaluator`
* Statystyki: `ChiSquareTest`, `Correlation.corr`, `Summarizer`

---

## 4) Pipeline’y i strojenie

* `Pipeline` (łańcuch Transformerów i Estimatorów)
* `ParamGridBuilder` + `CrossValidator` lub `TrainValidationSplit`
* Zapisywanie/ładowanie: `pipelineModel.save(...)`, `...load(...)`

---

## 5) Szkielet: minimalny pipeline (klasyfikacja)

```python
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.appName("mllib-starter").getOrCreate()

df = spark.read.csv("data.csv", header=True, inferSchema=True)

cat = ["k1","k2"]
num = ["x1","x2","x3"]
label = "y"

indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") for c in cat]
ohe = OneHotEncoder(inputCols=[f"{c}_idx" for c in cat],
                    outputCols=[f"{c}_oh" for c in cat])
assembler = VectorAssembler(
    inputCols=[*num, *[f"{c}_oh" for c in cat]],
    outputCol="features_raw"
)
scaler = StandardScaler(inputCol="features_raw", outputCol="features", withMean=True, withStd=True)

clf = GBTClassifier(featuresCol="features", labelCol=label, maxDepth=6)

pipe = Pipeline(stages=[*indexers, ohe, assembler, scaler, clf])
train, test = df.randomSplit([0.8, 0.2], seed=42)
model = pipe.fit(train)

pred = model.transform(test)
acc = MulticlassClassificationEvaluator(labelCol=label, predictionCol="prediction",
                                        metricName="accuracy").evaluate(pred)
print("Accuracy:", acc)

model.write().overwrite().save("models/gbt_pipeline")
```

---

## 6) Strojenie hiperparametrów (skrót)

```python
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

grid = (ParamGridBuilder()
        .addGrid(clf.maxDepth, [4, 6, 8])
        .addGrid(clf.maxIter, [50, 100])
        .build())

evaluator = BinaryClassificationEvaluator(labelCol=label, metricName="areaUnderROC")

cv = CrossValidator(estimator=pipe, estimatorParamMaps=grid,
                    evaluator=evaluator, numFolds=3, parallelism=2)

cv_model = cv.fit(train)
print("CV AUC:", evaluator.evaluate(cv_model.transform(test)))
```

---

## 7) Co zwykle wybrać na start (praktycznie)

* **Klasyfikacja binarna**: `LogisticRegression` (baseline), potem `RandomForestClassifier` / `GBTClassifier`.
* **Wieloklasowa**: `LogisticRegression` z `OneVsRest` lub `RandomForestClassifier`.
* **Regresja**: `LinearRegression` (baseline), potem `GBTRegressor`.
* **Klasteryzacja**: `KMeans` → sprawdź `K` (np. silhouette).
* **Rekomendacje**: `ALS`.

---

## 8) Na co uważać

* Ustaw `handleInvalid="keep"` w `StringIndexer`/`OneHotEncoder` przy danych realnych.
* Zawsze trzymaj wszystkie cechy w **jednej** kolumnie `features` (Vector).
* Rozdziel `train/test` **przed** strojenie CV.
* Pipelines zapisuj jako całość (łatwiejsze wdrożenia).
* MLlib jest głównie **CPU**; jeśli chcesz GPU, to przez **XGBoost4J-Spark** lub **RAPIDS** (osobne dodatki).

---

Jeśli powiesz, co chcesz zbudować (klasyfikacja/regresja/klastery/rekomendacje + format danych), ułożę Ci gotowy pipeline „pod klucz” z najlepszymi defaultami i metryką.


In [1]:
!nvidia-smi

Sun Oct 26 18:35:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.57                 Driver Version: 581.57         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   47C    P8              3W /   80W |     472MiB /   6144MiB |     20%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----